In [1]:
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
import numpy as np

In [2]:
# conectarse a bbdd mysql
with open ('password.txt') as f:
   pwd=str(f.readline()).strip()

conexion = pymysql.connect(host='localhost',
                           database = 'henry_pi',
                           user= 'root',
                           password=pwd)

cursor = conexion.cursor()

In [3]:
# se le pasa el nombre de una tabla contenida en una base de datos a la cual se esta conectado
# devuelve un dataframe
def extraer_datos(tabla:str):
    cursor.execute("SELECT * FROM {}".format(tabla))
    rows = cursor.fetchall()
    headers = [column[0] for column in cursor.description]
    df= pd.DataFrame(rows,columns=headers)
    return df


In [4]:
# extraer datos de bbdd
historico_velocidad_nacional=extraer_datos('historico_velocidad_nacional')
ingresos_trimestre=extraer_datos('ingresos_trimestre')
internet_accesos_velocidad_provincias=extraer_datos('internet_accesos_velocidad_provincias')
internet_penetracion_hogares=extraer_datos('internet_penetracion_hogares')
internet_tipo_nacional=extraer_datos('internet_tipo_nacional')
internet_tipo_provincias=extraer_datos('internet_tipo_provincias')
localidades_servicios=extraer_datos('localidades_servicios')
salarios_promedio=extraer_datos('salario_promedio_2023')

In [5]:
# unir los dataframes segun la estructura de los datos
nacionales = [historico_velocidad_nacional,ingresos_trimestre,internet_tipo_nacional]
provinciales = [internet_accesos_velocidad_provincias,internet_penetracion_hogares,internet_tipo_provincias]
locales=[localidades_servicios]

In [6]:
# deshacerse de algunas columnas innecesarias
historico_velocidad_nacional.drop(columns='trimestre_d',inplace=True)
ingresos_trimestre.drop(columns='periodo',inplace=True)

In [7]:
df_nacionales = reduce(lambda left,right:pd.merge(left,right, on=['año','trimestre'],how='outer'),nacionales) # uso reduce para hacer un join de los dataframes

In [8]:
internet_tipo_provincias['trimestre']=internet_tipo_provincias.trimestre.astype(int)
internet_accesos_velocidad_provincias.rename(columns={'otros':'otras_velocidades'},inplace=True)
#internet_tipo_provincias.drop(columns='total',inplace=True)

In [9]:
df_provinciales = reduce(lambda left,right:pd.merge(left,right,on=['año','trimestre','provincia'],how='outer'),provinciales)

In [10]:
df_locales = locales[0]

In [11]:
# latitud y longitud a  float
df_locales['latitud']=df_locales['latitud'].str.replace('..','.')
df_locales['latitud']=df_locales['latitud'].astype(float)


In [12]:
df_locales['longitud']=df_locales['longitud'].str.replace('..','.')
df_locales['longitud']=df_locales['longitud'].astype(float)
df_locales.rename(columns={'latitud':'latitude','longitud':'longitude'})

,provincia,partido,localidad,poblacion,adsl,cable_modem,dial_up,fibra_opt,satelital,wireless,telefonia_fija,3g,4g,link,latitude,longitude
0,BUENOS AIRES,Bahía Blanca,Villa Bordeau,0,NO,NO,NO,NO,NO,NO,NO,SI,SI,6056020,-38.647261,-62.320022
1,BUENOS AIRES,Bahía Blanca,Villa Espora,0,NO,NO,NO,NO,NO,NO,NO,NO,NO,6056020,-38.776507,-62.185183
2,BUENOS AIRES,Balcarce,Balcarce,38376,SI,SI,SI,SI,NO,SI,SI,SI,SI,6063010,-37.848278,-58.255167
3,BUENOS AIRES,Balcarce,Los Pinos,337,NO,NO,NO,NO,NO,SI,SI,NO,NO,6063020,-37.941206,-58.322592
4,BUENOS AIRES,Balcarce,Napaleofú,374,NO,NO,NO,SI,NO,SI,SI,SI,SI,6063030,-37.625498,-58.746186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,CHACO,Quitilipi,Quitilipi,24517,SI,NO,NO,SI,NO,SI,SI,SI,SI,22133010,-26.873207,-60.218524
4308,CHACO,Quitilipi,Villa El Palmar,879,NO,NO,NO,NO,NO,SI,NO,NO,SI,22133020,-26.455119,-60.164617
4309,CHACO,San Fernando,Barranqueras,54698,SI,SI,NO,SI,NO,SI,SI,SI,SI,22140010,-27.487774,-58.932742
4310,CHACO,San Fernando,Basail,1929,SI,SI,NO,NO,SI,SI,SI,SI,SI,22140020,-27.886866,-59.279100


In [13]:
df_locales.loc[df_locales['longitud']<-100,'longitud']=df_locales.longitud/1000
df_locales.loc[df_locales['latitud']<-100,'latitud']=df_locales.latitud/1000

In [14]:
# quitar caracteres especiales de provincias para georreferencias
mapping = {'á': 'a',
           'é': 'e',
           'í': 'i',
           'ó': 'o',
           'ú': 'u',
           'ñ': 'n',
           'D': 'd'}

df_provinciales.provincia.replace(mapping, regex=True,inplace=True)


In [15]:
df_provinciales['provincia'].unique()
# Santiago del Estero
# Santiago Del Estero

array(['Buenos Aires', 'Capital Federal', 'Catamarca', 'Chaco', 'Chubut',
       'Cordoba', 'Corrientes', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen',
       'Rio Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'Santiago del Estero', 'Tierra del Fuego', 'Tucuman'],
      dtype=object)

In [16]:
df_provinciales.head()

,año,trimestre,provincia,hasta_512kbps,512_kbps_1mbps,1mbps_6mbps,6mbps_10mbps,10mbps_20mbps,20mbps_30mbps,30mbps_x,otras_velocidades,total_x,accesos_100_hogares,banda_ancha_fija,dial_up,total_y
0,2022,3,Buenos Aires,29985.0,27709.0,290315.0,297915.00,267044.0,124.19,3618690.00,65821.0,4721670.0,78.11,4715469,6199,4721668
1,2022,3,Capital Federal,517.0,5742.0,34371.0,67829.00,51946.0,28692.00,1253100.00,105477.0,1547680.0,122.28,1545534,2145,1547679
2,2022,3,Catamarca,71.0,384.0,3107.0,5389.00,5099.0,3737.00,50298.00,2208.0,70293.0,65.33,70292,1,70293
3,2022,3,Chaco,461.0,987.0,16782.0,18938.00,8049.0,15828.00,79.39,3711.0,144146.0,43.86,144141,5,144146
4,2022,3,Chubut,109.0,1444.0,45707.0,30.94,34682.0,15309.00,17563.00,20024.0,165778.0,84.38,164874,904,165778


In [17]:
# corregir errores en los valores (venian con puntos mal colocados)
for i in df_provinciales:
    if (df_provinciales[i].dtype==float) & (i!='accesos_100_hogares'):
        floats=df_provinciales.loc[df_provinciales[i]%1 != 0,i]
        df_provinciales.loc[df_provinciales[i]%1 != 0,i]=floats*1000


In [18]:
# provincia a capitalize
salarios_promedio['provincia'] = salarios_promedio['provincia'].str.title()
df_salarios=salarios_promedio
df_salarios['provincia']=df_salarios['provincia'].str.replace('D','d')


In [19]:
df_locales['provincia'] = df_locales['provincia'].str.title()
df_locales['provincia'] = df_locales['provincia'].str.replace('D','d')
df_locales['localidad'] = df_locales['localidad'].str.title()


In [20]:
poblacion = pd.read_csv('datos_ordenados/poblacion_provincias.csv')
poblacion.dropna(inplace=True)

mapping ={
'NOA' : ['Jujuy','Salta','Tucuman','Catamarca','Santiago del Estero','La Rioja'],
'NEA' : ['Formosa','Chaco','Misiones','Corrientes','Santa Fe','Entre Rios'],
'Centro' : ['Cordoba'],
'Cuyo' : ['San Juan','San Luis','Mendoza'],
'Buenos Aires' : ['Buenos Aires','Capital Federal'],
'Patagonia' : ['La Pampa','Neuquen','Rio Negro','Chubut','Santa Cruz','Tierra del Fuego']
}

def get_region(row):
    for region, provincias in mapping.items():
        if row['provincia'] in provincias:
            return region
    return 'Other'

poblacion['region'] = poblacion.apply(get_region, axis=1)



In [21]:
df_nacionales.to_csv('datos_ordenados/nacionales.csv',index=False)
df_provinciales.to_csv('datos_ordenados/provinciales.csv',index=False)
df_locales.to_csv('datos_ordenados/locales.csv',index=False)
df_salarios.to_csv('datos_ordenados/salarios_provincias.csv',index=False)
poblacion.to_csv('datos_ordenados/poblacion_provincias.csv',index=False)

In [22]:
df_salarios

,provincia,promedio_salario
0,Buenos Aires,345171.0516
1,Capital Federal,406617.4563
2,Catamarca,195569.2105
3,Chaco,181364.6624
4,Chubut,321039.8417
5,Cordoba,237196.3695
6,Corrientes,179483.2278
7,Entre Rios,206123.8398
8,Formosa,173632.5463
9,Jujuy,208999.7906
